In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

main_df = pd.read_csv("../data/raw/initial_games_data.csv")
platform_df = pd.read_csv("../data/raw/platform_game_data.csv")
steam_games_df = pd.read_csv("../data/raw/games.csv", index_col=False)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(f"Main Dataset: {main_df.shape[0]} Second dataset: {platform_df.shape[0]}")

Main Dataset: 67571 Second dataset: 50872


C:\Users\lotec\AppData\Local\Temp\ipykernel_23512\2165180709.py:7: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  steam_games_df = pd.read_csv("../data/raw/games.csv", index_col=False)


In [83]:
steam_games_df = steam_games_df[["AppID", "Categories", "Genres", "Tags"]]
steam_games_df = steam_games_df.rename(columns={"AppID": "app_id"})

In [84]:
main_df = main_df.drop(columns=["Unnamed: 0"]) # Drop unnecessary index column
main_df = main_df.iloc[:, :4] # Keep only the first 4 columns
# Keeps the first occurrence of each game via link
main_df = main_df.drop_duplicates(subset='link', keep='first')

main_df["app_id"] = main_df["link"].str.extract(r'/app/(\d+)/')
main_df['app_id'] = main_df['app_id'].astype(int) 
main_df = main_df.drop(columns=['link'])


cols = ['app_id'] + [c for c in main_df.columns if c != 'app_id']
main_df = main_df[cols]
# print(main_df.info())
# print(f"Main Dataset: {main_df.shape[0]} Second dataset: {platform_df.shape[0]}")
# print(platform_df.info())
platform_df = platform_df.drop(columns=["rating", "positive_ratio", "user_reviews", "price_final", "discount", "steam_deck"])
# print(platform_df.info())
merged_df = pd.merge(main_df, platform_df, on='app_id')
# print(merged_df.info())

# print(merged_df.shape[0])
# Find rows where release and date_release differ
differences = merged_df[merged_df["release"] != merged_df["date_release"]]

# Show first 20 differences
# print(differences[["app_id", "release", "date_release"]].head(20))

# Count total differences
# print("Number of differing rows:", len(differences))


game_dupes = merged_df["game"].duplicated().sum()
title_dupes = merged_df["title"].duplicated().sum()

# print(f"Duplicate game names: {game_dupes}")
# print(f"Duplicate title names: {title_dupes}")

merged_df = merged_df.drop(columns=["game", "date_release"])
# print(merged_df.info())
# print(merged_df.head(10))


merged_final = pd.merge(
    merged_df,
    steam_games_df,
    on="app_id",
    how="inner" 
)

# print(merged_final.info())
print(merged_final.head(10))

matched = merged_final["Categories"].notna().sum()
print(f"Games with category info added: {matched}")

print(merged_final.loc[merged_final["app_id"] == 379720])

    app_id     release  peak_players                             title   win    mac  linux  price_original          Categories                                             Genres                                               Tags
0  2231450  2023-01-26          4529                       Pizza Tower  True  False  False            0.00       Tour De Pizza  Single-player,Steam Achievements,Full controll...                                       Action,Indie
1  2050650  2023-03-24        168191                   Resident Evil 4  True  False  False            0.00    CAPCOM Co., Ltd.  Single-player,Steam Achievements,Full controll...                                   Action,Adventure
2  2324650  2023-03-31         15543  The Murder of Sonic the Hedgehog  True   True  False            0.00                SEGA                                      Single-player               Casual,Free to Play,Indie,Simulation
3  2263010  2023-03-28          1415                Pineapple on pizza  True   True 